In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [3]:
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))

['unripe', 'ripe', 'rotten', 'overripe']


In [4]:
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification'
CATEGORIES =["overripe","ripe","rotten","unripe"]
IMG_SIZE = 100

In [5]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [6]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [7]:
X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE * IMG_SIZE * 3) # Düzleştirme işlemi
y = np.array(y)

In [8]:
# Veriyi bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [9]:
# Etiketleri one-hot kodlama
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
# Modeli oluşturma
model = Sequential()
model.add(Dense(1024, input_shape=(IMG_SIZE * IMG_SIZE * 3,), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax')) # Çıkış katmanı 4 nöronlu olmalı ve 'softmax' kullanmalı


In [12]:
# Modeli derleme
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [13]:
# Modeli eğitme
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
304/304 [==============================] - 119s 383ms/step - loss: 575.9465 - accuracy: 0.4315 - val_loss: 26.5025 - val_accuracy: 0.4634
Epoch 2/10
304/304 [==============================] - 109s 359ms/step - loss: 13.5443 - accuracy: 0.5857 - val_loss: 8.6525 - val_accuracy: 0.5533
Epoch 3/10
304/304 [==============================] - 110s 362ms/step - loss: 9.2794 - accuracy: 0.5771 - val_loss: 8.0103 - val_accuracy: 0.4208
Epoch 4/10
304/304 [==============================] - 109s 358ms/step - loss: 4.6491 - accuracy: 0.6003 - val_loss: 5.5250 - val_accuracy: 0.5607
Epoch 5/10
304/304 [==============================] - 121s 397ms/step - loss: 4.3234 - accuracy: 0.5957 - val_loss: 3.9774 - val_accuracy: 0.5533
Epoch 6/10
304/304 [==============================] - 116s 381ms/step - loss: 2.5523 - accuracy: 0.6199 - val_loss: 0.9818 - val_accuracy: 0.6960
Epoch 7/10
304/304 [==============================] - 116s 383ms/step - loss: 0.9320 - accuracy: 0.6989 - val_loss: 1.28

In [14]:
# Test seti üzerinde değerlendirme
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

85/85 [==============================] - 3s 36ms/step


In [15]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")


In [17]:
# Sonuçları gösterme
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy:  0.5890207715133531
Precision:  0.6050741046217956
Recall:  0.5890207715133531
F1 Score:  0.5423196183624076

Confusion Matrix:
 [[ 40 191 298  16]
 [ 23 436 320  12]
 [  4  66 779  53]
 [  0  64  61 333]]

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.07      0.13       545
           1       0.58      0.55      0.56       791
           2       0.53      0.86      0.66       902
           3       0.80      0.73      0.76       458

    accuracy                           0.59      2696
   macro avg       0.63      0.55      0.53      2696
weighted avg       0.61      0.59      0.54      2696

